# Harmony EOSS L2SS API Tutorial

## Before you start
Before you beginning this tutorial, make sure you have an account in the Earthdata Login UAT environment, which 
will be used for this notebook by visiting [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov).
These accounts, as all Earthdata Login accounts, are free to create and only take a moment to set up.

## Set Up Authentication

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [1]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


Now call the above function to set up Earthdata Login for subsequent requests

In [2]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

Please provide your Earthdata Login credentials to allow data access
Your credentials will only be passed to uat.urs.earthdata.nasa.gov and will not be exposed in Jupyter


Username: mgangl
 ·········


## Build the EOSS Root URL

Next we will build a URL for the EOSS service for a given granule.  To get data using the service, you need 
a [CMR UAT](https://cmr.uat.earthdata.nasa.gov) collection ID for a supported collection and the ID of a
granule within that collection.

By convention, all Harmony services are accessed through `<harmony_root>/<collection_id>/<service_name>`

In [13]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
config = {
    'collection_id': 'C1225996408-POCUMULUS',
    'eoss_version': '0.1.0'
}
eoss_collection_root = harmony_root+'/{collection_id}/eoss/{eoss_version}/items/'.format(**config)
print(eoss_collection_root)

https://harmony.uat.earthdata.nasa.gov/C1225996408-POCUMULUS/eoss/0.1.0/items/


### Find a granule for subsetting

Below we call out a specific granule (G1226018995-POCUMULUS) on which we will use the podaac L2 subsetter. Finding this information would complicate the tutorial- but po.daac has a tutorial available for using the CMR API to find collections and granules of interest. Please see the following tutorial for that information:

PODAAC_CMR.ipynb


##  Subset of a PO.DAAC Granule

We can now build onto the root URL in order to actually perform a transformation.  The first transformation is a subset of a selected granule.  _At this time, this requires discovering the granule id from CMR_.  That information can then be appended to the root URL and used to call Harmony with the help of the `request` library.

Above we show how to find a granule id for processing.

**Notes:**
  The L2 subsetter current streams the data back to the user, and does not stage data in S3 for redirects. This is functionality we will be adding over time.
  It doesn't work with URS backed files, which is coming in the next few weeks
  it only works on the show dataset, but 
    

In [17]:
import shutil 

granuleConfig = {
    'granule_id' : 'G1226018995-POCUMULUS',
}
spatialSubsetConfig = {
    'west' : '-90',
    'south' : '-45',
    'east' : '90',
    'north' : '45'
}
eoss_var_subset_url = eoss_collection_root+'{granule_id}/?bbox={west},{south},{east},{north}'.format(**granuleConfig, **spatialSubsetConfig)

print('Request URL', eoss_var_subset_url)

with request.urlopen(eoss_var_subset_url) as response, open('temp.nc', 'wb') as out_file:
    print('Stream response, not saved to S3:', response.geturl())
    print('Content Size:', response.headers['Content-length'])
    shutil.copyfileobj(response, out_file)
    print("Downloaded request to temp.nc")

Request URL https://harmony.uat.earthdata.nasa.gov/C1225996408-POCUMULUS/eoss/0.1.0/items/G1226018995-POCUMULUS/?bbox=-90,-45,90,45
Stream response, not saved to S3: https://harmony.uat.earthdata.nasa.gov/C1225996408-POCUMULUS/eoss/0.1.0/items/G1226018995-POCUMULUS/?bbox=-90,-45,90,45
Content Size: 6381633
Downloaded request to temp.nc


## Same operation with OGC API

The PO.DAAC Subsetter didn't need any updates for this to work.

In [15]:
bboxSubsetConfig = {
    'collection_id': 'C1225996408-POCUMULUS',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': 'G1226018995-POCUMULUS',
    'lat': '(-45.75:45)',
    'lon': '(-90:90)'
}
bbox_url = harmony_root+'/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?granuleid={granuleid}&subset=lat{lat}&subset=lon{lon}'.format(**bboxSubsetConfig)
print('Request URL', bbox_url)


Request URL https://harmony.uat.earthdata.nasa.gov/C1225996408-POCUMULUS/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleid=G1226018995-POCUMULUS&subset=lat(-45.75:45)&subset=lon(-90:90)


In [16]:
with request.urlopen(bbox_url) as response, open('ogc_temp.nc', 'wb') as out_file:
    print('Content Size:', response.headers['Content-length'])
    shutil.copyfileobj(response, out_file)
    print("Downloaded request to ogc_temp.nc")

Content Size: 6434121
Downloaded request to ogc_temp.nc
